# Analyse van AskDelphi Relations & Tags endpoint
Dit notebook ondersteunt het verkennen, analyseren en valideren van de Relations & Tags functionaliteit binnen AskDelphi.  
De module biedt helpermethoden voor het toevoegen, beheren en inspecteren van relaties en tags die aan AskDelphi‑topics gekoppeld zijn.  
Het doel van dit notebook is om:

- inzicht te krijgen in de datastructuren en API‑interacties rondom relations & tags
- voorbeelddata te analyseren en patronen te ontdekken
- helpermethoden te testen en gedrag te evalueren
- experimentele of onderzoeksgerichte analyses uit te voeren ter verbetering van de contentstructuur in AskDelphi

Dit notebook vormt daarmee een flexibel startpunt voor verdere exploratie of debugging van de Relations & Tags endpoint.

### Importeren benodigde Python Libs

In [42]:
import dotenv
import logging
import sys
import os
import uuid
from pprint import pprint
from pathlib import Path
from src.askdelphi_client import AskDelphiClient

ROOT = Path().resolve().parent
sys.path.append(str(ROOT))
env_path = ROOT / ".env"

dotenv.load_dotenv(env_path)

logging.disable(sys.maxsize)

os.getenv("ASKDELPHI_PORTAL_CODE")

'S2RBZV-755ZA4'

### Connectie met AskDelphi opzetten

In [48]:
client = AskDelphiClient()
client.authenticate()

True

### Helper functies (@Todo vervangen door API functionaliteit)

In [53]:
def get_topic_types() -> dict:
    """ Haalt de beschikbare AskDelphi-topictype IDs op.
    Returns:        
    dict: Mapping met topictype
    ‑IDs voor:            - "action"            - "task"            - "digitale coach procespagina"    """
    topic_type_map = {}
    try:
        design = client.get_content_design()

        topic_types = design.get("topicTypes", [])
        # print(f"Found {len(topic_types)} topic types:\n")

        for i, tt in enumerate(topic_types[:100]):  # Show first 100
            if tt.get("title").lower() in ["action", "task", "digitale coach procespagina"]:
                # print(f"  {i+1}. {tt.get('title', 'Unknown')}")
                # print(f"     ID: {tt.get('key', 'N/A')}")
                # print(f"     Namespace: {tt.get('namespace', 'N/A')}")
                # print()

                topic_type_map[tt.get("title")] = tt.get("key")

        if len(topic_types) > 100:
            print(f"  ... and {len(topic_types) - 100} more")

    except Exception as e:
        print(f"FAILED: {e}")

    # print(f"topic type map:\n")
    # print(topic_type_map)

    return topic_type_map

def add_topic(topic_id: str, topicTitle: str, topicTypeId: str):
    """Voeg een topic toe van meegegeven topic type."""
    endpoint = "v4/tenant/{tenantId}/project/{projectId}/acl/{aclEntryId}/topic"
    return client._request(
        "POST",
        endpoint,
        json_data={
            "topicId": topic_id,
            "topicTitle": topicTitle,
            "topicTypeId": topicTypeId
        }
    )

def checkout(topic_id: str) -> str:
    """Checkout een topic voor bewerking en return ."""
    endpoint = "v1/tenant/{tenantId}/project/{projectId}/acl/{aclEntryId}/topic/{topic_id}/workflowstate"
    result = client._request("POST", endpoint,
        json_data={
            "action": 1
        })
    topicVersionId = result["topicVersionId"]
    return topicVersionId

### Uitwerking API client code

In [50]:
def add_relation(self, topic_id: str, topic_version_id: str, relation_type_id: str, target_topic_ids: list):
    """Voeg een relatie toe van dit topic naar andere topics."""
    endpoint = "v2/tenant/{tenantId}/project/{projectId}/acl/{aclEntryId}/topic/{topic_id}/topicVersion/{topic_version_id}/relation"
    return self.session.post(
        endpoint,
        json={
            "relationTypeId": [3],
            "sourceTopicId": topic_id,
            "targetTopicIds": target_topic_ids,
        }
    )

def add_topic_with_relation(client: AskDelphiClient, topicId: str, topicTitle: str, topicTypeId: str, parentTopicId: str, parentTopicRelationTypeId: str, parentTopicVersionId: str):
    """Voeg een topic met een relatie naar andere topic toe.  
    Args:        
    - topicId (str): ID van het doel-topic.        
    - topicTitle (str): Titel van het doel-topic.        
    - topicTypeId (str): ID van het topictype.        
    - parentTopicId (str): ID van het bron-topic.
    - parentTopicRelationTypeId (str): ID van het relatietype.        
    - parentTopicVersionId (str): Versie-ID van het bron-topic."""
    endpoint = "v4/tenant/{tenantId}/project/{projectId}/acl/{aclEntryId}/topic"
    return client._request(
        "POST",
        endpoint,
        json_data={
            "topicId": topicId,
            "topicTitle": topicTitle,
            "topicTypeId": topicTypeId,
            "parentTopicId": parentTopicId,
            "parentTopicRelationTypeId": parentTopicRelationTypeId,
            "parentTopicVersionId": parentTopicVersionId
        }
    )

def add_tag(self, topic_id: str, topic_version_id: str, hierarchy_topic_id: str, hierarchy_node_id: str):
    """Voeg een tag toe aan een topic."""
    endpoint = f"v2/tenant/{{tenantId}}/project/{{projectId}}/acl/{{aclEntryId}}/topic/{topic_id}/topicVersion/{topic_version_id}/tag"
    return self.session.post(
        endpoint,
        json={
            "tags": [
                {
                    "hierarchyTopicId": hierarchy_topic_id,
                    "hierarchyNodeId": hierarchy_node_id,
                }
            ]
        },
    )


In [51]:
DIGICOACH_NAME = "Digicoach test arie 3feb"
TASK_NAME = "Taak test 3feb"

In [61]:
# Create Digicoach
topic_id = str(uuid.uuid4())
topic_type_map = get_topic_types()
topicTypeId = topic_type_map["Digitale Coach Procespagina"]

response = add_topic(topic_id, DIGICOACH_NAME, topicTypeId)
print(topic_id)

9c17effd-1558-4f82-9824-74960cf34c6d


In [ ]:
"""Checkout een topic voor bewerking en return ."""
endpoint = "v1/tenant/{tenantId}/project/{projectId}/acl/{aclEntryId}/topic/{topic_id}/workflowstate"
result = client._request(
    "POST", 
    endpoint,
    json_data={"action": 1})
topicVersionId = result["topicVersionId"]
print(topicVersionId)

Exception: API POST v1/tenant/{tenantId}/project/{projectId}/acl/{aclEntryId}/topic/{topic_id}/workflowstate failed!

  Status Code: 400 Bad Request
  URL: https://edit.api.askdelphi.com/v1/tenant/0be6d42b-c278-44e6-888e-ba122840d690/project/397296f6-20dd-45cd-8459-250db2725140/acl/4ecd88f2-979b-4fb0-a95d-175d499bc375/topic/%7Btopic_id%7D/workflowstate
  Content-Type: application/problem+json; charset=utf-8
  Response (JSON): {
    "type": "https://tools.ietf.org/html/rfc9110#section-15.5.1",
    "title": "One or more validation errors occurred.",
    "status": 400,
    "errors": {
        "topicId": [
            "The value '{topic_id}' is not valid."
        ]
    },
    "traceId": "00-388d6039964e0f97b326031e02bb1c9c-bc71808f3ed59d04-00"
}

Troubleshooting:

In [30]:
# get relationship ID for task
task_relation_id = ""
endpoint = f"/v2/tenant/{{tenantId}}/project/{{projectId}}/acl/{{aclEntryId}}/topic/{topic_id1}/topicVersion/{topicVersionId}/allowedrelations"
result = client._request("GET", endpoint,
                json_data={}
                )
for relation in result["topicAllowedRelations"]:
    if relation["topicTypeName"] == "Taak":
        task_relation_id = relation["relationTypeId"]
        break

task_relation_id



'1de7ca2f-d566-4245-96f6-f4cc56434685'

In [31]:
topic_id2 = str(uuid.uuid4())

result_2 = client._request(
    "POST",
    "v4/tenant/{tenantId}/project/{projectId}/acl/{aclEntryId}/topic",
    json_data={
        "topicId": topic_id2,
        "topicTitle": TASK_NAME,
        "topicTypeId": topic_type_map["Task"],
        "parentTopicId": topic_id1,
        "parentTopicRelationTypeId": task_relation_id,
        "parentTopicVersionId": topicVersionId
        }
)